# Text Repräsentationen
In diesem Teil des Labors wird es um eine Einfürung in Text Repräsentationen gehen. 

Wie Ihr vermutlich schon wisst, repräsentieren Computer Daten in Binärcode. Um Euch das Problem, das Computer mit Texten haben aufzuzeigen, hat euch der Computer eine Nachricht hinterlassen, die ihr Entschlüsseln müsst... 

In [1]:
binary_message = "01001001 00100000 01110011 01110000 01100101 01100001 01101011 00100000 01100010 01101001 01101110 01100001 01110010 01111001 00100000 01101111 01101110 01101100 01111001 00100001 00001010 01001001 00100000 01101100 01101001 01101011 01100101 00100000 01101110 01110101 01101101 01100010 01100101 01110010 01110011 00101100 00100000 01101110 01101111 01110100 00100000 01110100 01100101 01111000 01110100 00101110"

In [2]:
binary_chars = binary_message.split()
message = ""
for binary_char in binary_chars:
    binary_char_integer = int(binary_char, 2)
    character = chr(binary_char_integer)
    message += character
print(message)

I speak binary only!
I like numbers, not text.


Um Maschinen Text verständlich zu machen, müssen wir die Texte in eine andere Repräsentation bringen. Eine Kurze Einführung bieten die beiden folgenden Warm Up Aufgaben.

## Tokenization
Eine der ersten elementaren Schritte, um eine natürliche Sprache mit einem Computer zu verarbeiten ist das Aufteilen eines Fließtextes in kleinere Teile. Meist werden Texte direkt in sogenannte Tokens geteilt (Tokenization). Ein Token ist oftmals ein einzelnes Wort, es kann aber auch aus mehreren Wörtern bestehen, wenn diese zusammen für eine Entität stehen, z.B. die Stadt „New York“. Neben der Aufteilung in Tokens ist es auch möglich, einen Text zuerst in Sätze zu splitten (Sentence Tokenization). Welche Kombination der Methoden angewendet wird, hängt von der Aufgabe ab. 

### Warm Up 1: Tokenizer mit Regulären Ausdrücken

Die erste kleine Aufgabe besteht darin mit einem regulären Ausdruck ein Wort-Tokenizer zu bauen. (Diese Aufgabe sollte nicht mehr als 5 min. dauern) Der Tokenizer muss auch nicht perfekt sein. 

> Eine kurze Auffrischung über Reguläre Ausdrücke findet sich in https://regexr.com/

In [3]:
import re
text="Die Ente lacht und quakt."

tokenizer_regex=re.compile(r'\s')
tokenizer_regex.split(text)

['Die', 'Ente', 'lacht', 'und', 'quakt.']

Das Grundprinzip eines Tokenizers sollte euch nun klar sein. In der Praxis empfiehlt es sich allerdings, auf bestehende Tokenizer-Implementierungen zurückzugreifen.
Im Python-Umfeld ist https://spacy.io/api/tokenizer oder https://www.nltk.org/api/nltk.tokenize.html sehr empfehlenswert.

# Encoding

Der nächste Schritt, um mit Texten Maschinelles Lernen zu betreiben, ist das Umwandeln der Wörter in eine numerissche Repräsentation. In der nächsten Warm-Up Aufgabe wird es darum um Bag of Words gehen.

## Warm Up 2: Bag of Words mit scikit-learn
Ein Ansatz um Texte bzw. Tokens als Zahlen zu represäntieren ist ein Bag of Words. Hier wird jeder Text als Vektor dargestellt (Länge: Länge des Vokabulars). Jeder Eintrag im Vektor steht für ein Wort im Vokabular.

In der nächsten Aufgabe wollen wir mit dem vorgegebenen Textkorpus, den Satz **"Die Ente singt und quakt."** in einen bag-of-words-encoded Vector umwandeln.
Dazu bietet sich der `CountVectorizer` an (siehe [Dokumentation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)).

> Kleiner Tipp: Der `CountVectorizer` übernimmt diesmal die Tokenisierung.

In [4]:
from sklearn.feature_extraction.text import CountVectorizer

corpus = ["Die Ente lacht und quakt.",
          "Die Ente singt und tanzt."]
vectorizer = CountVectorizer()

bag_of_words_encoded_sentence = vectorizer.fit_transform(corpus).toarray()
print(bag_of_words_encoded_sentence)

[[1 1 1 1 0 0 1]
 [1 1 0 0 1 1 1]]


Im  nächsten Schritt gebt die Länge der Satzrepräsentation aus. 

Wie verändert sich die Länge, wenn sich der Textkorpus vergrößert. Was sind die Vorteile und Nachteile von Bag of Words?

In [5]:
len(bag_of_words_encoded_sentence[0])

7

## Word Embeddings

Natürliche Sprachen bieten uns mannigfaltige Möglichkeiten, dieselben Inhalte auf unterschiedliche Art und Weise auszudrücken. Wenn wir mit Texten arbeiten, reicht es daher in der Regel nicht aus, nur die Anzahl und das Vorkommen von bestimmten Wörtern in Texten zu betrachten, weil neben des bloßen Vorkommen eines Wortes auch dessen Anordnung und dessen Bedeutung eine Rolle spielt. Als zusätzliche und wichtige Ebene kommt hier also die Semantik ins Spiel.

Die Abbildung von Wörtern auf Vektoren erlaubt es uns, mit diesen zu rechnen und zum Beispiel Distanzen oder Ähnlichkeiten zu bestimmen. Embeddings haben den Vorteil, dass sie eine Dimensionsreduktion mit sich bringen und semantische Embeddings sorgen darüber hinaus dafür, dass "verwandte" Wörter einen geringen Abstand voneinander haben.

Wir wollen uns im Folgenden zunächst mit Word2Vec beschäftigen und eine einfache Version des CBOW-Ansatzes selbst implementieren.

Danach schauen wir uns Gensim als Bibliothek für Word-Embedding-Modelle an und werfen einen Blick auf Evaluationsmethoden für Embeddings sowie die ihnen inhärenten Biase.

## Aufgabe 1: Word2Vec CBOW
> You shall know a word by the company it keeps.
>
> -- <cite>J. R. Firth</cite>

Auf dem oben zitierten Prinzip beruhen die beiden als Word2Vec bekannt gewordenen Modelle CBOW und Skip Gram, die 2013 von [Tomas Mikolov et al.](https://arxiv.org/abs/1301.3781) bei Google entwickelt wurden.
Erstgenanntes Modell werden wir im Folgenden in einer einfachen Form selbst implementieren.

#### 1.1 Trainingsdaten
Die Beschaffung und Aufbereitung von Trainingsdaten ist ein wichtiger Schritt in jeder NLP-Pipeline. Jetzt drücken wir uns mal davor und greifen auf einen Datensatz zu, den wir schonmal vorarb für Euch vorbereitet haben. Wir haben uns entschieden Alice im Wunderland [Gutenberg Project Alice im Wunderland](https://www.gutenberg.org/cache/epub/19778/pg19778.txt) zu nutzen um Word Embeddings zu trainieren. Der Datensatz beinhaltet die Sätze aus aus der Geschichte. 

Der vorverarbeitete Datensatz ist als pickle abgespeichert und findet sich in [hier](https://drive.google.com/file/d/1RfCivF-wHf33S7TMxjpT78923ADXreMi/view). Wir werden den googledrivedownloader nutzen, um die Datei zu laden. Ladet den Datensatz mit Pickle als Testdatensatz aus.

In [6]:
from google_drive_downloader import GoogleDriveDownloader as gdd

gdd.download_file_from_google_drive(file_id='1RfCivF-wHf33S7TMxjpT78923ADXreMi',
                                    dest_path='./download/alice_sentences.pkl',
                                    unzip=False,
                                    overwrite=True)

In [7]:
import pickle

file = open('./download/alice_sentences.pkl', 'rb')
data_sample = pickle.load(file)

### 1.2 Datenvorbereitung
Wir haben einen tokenisierten Datensatz, der aus Listen von Wörtern besteht. Jede Liste repräsentiert einen Satz. Unser Modell soll aber hinterher mit Zahlen hantieren und zwar entweder mit Wortindizes, die jedes Wort im Vokabular über einen eindeutige Nummer referenzierbar machen, oder mit One-hot-Vektoren, die als Labels dienen, mit denen der tatsächliche Output des Modells verglichen werden kann.

In [8]:
print(data_sample[:10])

[['Erstes', 'Kapitel', 'Hinunter', 'in', 'den', 'Kaninchenbau', '.'], ['Alice', 'fing', 'an', 'sich', 'zu', 'langweilen', ';', 'sie', 'saß', 'schon', 'lange', 'bei', 'ihrer', 'Schwester', 'am', 'Ufer', 'und', 'hatte', 'nichts', 'zu', 'thun', '.'], ['Das', 'Buch', ',', 'das', 'ihre', 'Schwester', 'las', ',', 'gefiel', 'ihr', 'nicht', ';', 'denn', 'es', 'waren', 'weder', 'Bilder', 'noch', 'Gespräche', 'darin', '.'], ['»', 'Und', 'was', 'nützen', 'Bücher', ',', '«', 'dachte', 'Alice', ',', '»', 'ohne', 'Bilder', 'und', 'Gespräche?«', 'Sie', 'überlegte', 'sich', 'eben', ',', '(', 'so', 'gut', 'es', 'ging', ',', 'denn', 'sie', 'war', 'schläfrig', 'und', 'dumm', 'von', 'der', 'Hitze', ',', ')', 'ob', 'es', 'der', 'Mühe', 'werth', 'sei', 'aufzustehen', 'und', 'Gänseblümchen', 'zu', 'pflücken', ',', 'um', 'eine', 'Kette', 'damit', 'zu', 'machen', ',', 'als', 'plötzlich', 'ein', 'weißes', 'Kaninchen', 'mit', 'rothen', 'Augen', 'dicht', 'an', 'ihr', 'vorbeirannte', '.'], ['Dies', 'war', 'grade',

In [9]:
from collections import OrderedDict

# Beim Mapping von Wörtern zu IDs und umgekehrt sollte eine reproduzierbare Reihenfolge sichergestellt werden,
# um das Modell später weitertrainieren und die Embedding-Matrix interpretieren zu können.
# Diese Datenstruktur kann, aber muss nicht, als Basis dienen.
unique_words = OrderedDict.fromkeys(list(set().union(*[set(sub_list) for sub_list in data_sample])))

# Mapping von Wort zu ID
word2id = {}
for idx, key in enumerate(unique_words):
    word2id[key] = idx
word2id['eos'] = len(unique_words)

# Mapping von ID zu Wort
id2word = dict((v,k) for k,v in word2id.items())

# Unser Data Sample, aber mit IDs statt Wörtern 
# [['der', 'hund', 'der', 'bellt'], ['die', 'katz', 'miaut']] => [[0, 1, 0, 2], [3, 4, 5]]
numeric_docs = [[word2id[w] for w in doc] for doc in data_sample]

print('Word to id sample:', list(word2id.items())[:10], '\n')
print('Id to word sample:', list(id2word.items())[:10], '\n')
print('Documents as lists of integers:', numeric_docs[0][:10])


Word to id sample: [('hier?«', 0), ('förmlich', 1), ('hervorrief', 2), ('gemüthlich', 3), ('Versprechen', 4), ('Usurpation', 5), ('Pfeffer', 6), ('hineinkommen?«', 7), ('Rauschen', 8), (',', 9)] 

Id to word sample: [(0, 'hier?«'), (1, 'förmlich'), (2, 'hervorrief'), (3, 'gemüthlich'), (4, 'Versprechen'), (5, 'Usurpation'), (6, 'Pfeffer'), (7, 'hineinkommen?«'), (8, 'Rauschen'), (9, ',')] 

Documents as lists of integers: [4295, 1098, 3293, 1607, 1694, 2677, 880]


Wir halten einige wichtige Parameter für unser Modell fest. Die Größe des Kontextfensters sowie die Länge der Embeddingvektoren können nach Bedarf angepasst werden. Für unsere Demo wählen wir kleine Werte.

In [10]:
vocabulary_size = len(unique_words) + 1
embedding_size = 50 # Länge der Embeddingvektoren
window_size = 2 # Größe des Kontextfensters. Wird nach rechts und links angewandt. Gesamter Kontext hier also 4 Wörter.

print('Vocabulary Size:', vocabulary_size)

Vocabulary Size: 4400


### 1.3 Generator
Um nicht alle Trainingsdaten auf einmal im Speicher halten zu müssen, schreiben wir uns eine Generatorfunktion, die Batches einer frei wählbaren Größe zurückgibt. Unser Ansatz ist dennoch nicht völlig speicherschonend, weil wir uns die Datengrundlage für die Generierung dieser Batches, nämlich die Integerlisten in `numeric_docs` sehr wohl im Speicher vorhalten. Darüber sehen wir aber großzügig hinweg.

Die Generatorfunktion erzeugt zwei numpy-Arrays der Länge `batch_size`, von denen das eine Listen mit Indizes der Kontexwörter enthält, die der Embedding-Layer als Eingabe erwartet, und das andere die zugehörigen One-Hot-Encodings der Mittelwörter.

Fiktives und vereinfachtes Beispiel:
<pre><code>* Fenstergröße: 1
* Batch-Size: 2
* Wortindizes: 'die': 0, 'ente': 1, 'lacht': 2, 'und': '3, 'quakt': 4 (und damit Vokabulargröße 5)
* Korpus (Auszug): [['die', 'ente', 'lacht', 'und', 'quakt'], ...]


=> Rückgabe: [[0, 2], [1, 3]], [[0, 1, 0, 0, 0], [0, 0, 1, 0, 0]]
</code></pre>

In [11]:
print(numeric_docs[:3])

[[4295, 1098, 3293, 1607, 1694, 2677, 880], [878, 482, 3633, 226, 1908, 2100, 1129, 2643, 4379, 4318, 2577, 239, 3497, 2081, 4381, 1724, 2613, 2572, 3700, 1908, 1881, 880], [1713, 1307, 9, 238, 3564, 2081, 767, 9, 3743, 3456, 4109, 1129, 337, 665, 1709, 1461, 2350, 656, 4398, 4322, 880]]


In [16]:
from keras.preprocessing import sequence
from keras.utils import np_utils
import numpy as np
import tensorflow as tf

def generate_context_word_batches(corpus, window_size, vocab_size, batch_size):
    X = []
    Y = []
    corpus = [item for sublist in corpus for item in sublist]
    print(f"vocab size: {vocab_size}")
    print(f"length of corpus after flatten: {len(corpus)}")
    current_size = 0
    while True:
        for idx, word in enumerate(corpus):
            if current_size != batch_size:
                context = []
                target_word = idx
                for i in range(target_word - window_size, target_word + window_size + 1):
                    if 0 <= i < len(corpus) and i != target_word:
                        context.append(corpus[i])
                X.append(np.array(context))

                oh_words = np.zeros(vocab_size)
                oh_words[corpus[target_word]] = 1
                Y.append(np.array(oh_words))
                
                current_size += 1
            else:
                # zwei numpy arrays
                current_size = 0
                X = tf.keras.utils.pad_sequences(X, window_size*2, value=vocabulary_size-1)
                yield (np.array(X), np.array(Y))
                X = []
                Y = []


In [17]:
# Schneller Test
test_batch_size = 3
test_window_size = 4

batch_gen = generate_context_word_batches(corpus=numeric_docs, window_size=test_window_size, vocab_size=vocabulary_size, batch_size=test_batch_size)
for i in range(0, 3): 
    x, y = next(batch_gen)
    print(f"x: {x}")
    print(f"y: {y}")
    for j in range(0, test_batch_size):
        print('Context (X):', [id2word[w] for w in x[j]], '-> Target (Y):', id2word[np.argwhere(y[j])[0][0]])  

vocab size: 4400
length of corpus after flatten: 31916
x: [[4399 4399 4399 4399 1098 3293 1607 1694]
 [4399 4399 4399 4295 3293 1607 1694 2677]
 [4399 4399 4295 1098 1607 1694 2677  880]]
y: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Context (X): ['eos', 'eos', 'eos', 'eos', 'Kapitel', 'Hinunter', 'in', 'den'] -> Target (Y): Erstes
Context (X): ['eos', 'eos', 'eos', 'Erstes', 'Hinunter', 'in', 'den', 'Kaninchenbau'] -> Target (Y): Kapitel
Context (X): ['eos', 'eos', 'Erstes', 'Kapitel', 'in', 'den', 'Kaninchenbau', '.'] -> Target (Y): Hinunter
x: [[4295 1098 3293 1607 2677  880  878  482]
 [1098 3293 1607 1694  880  878  482 3633]
 [3293 1607 1694 2677  878  482 3633  226]]
y: [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
Context (X): ['Erstes', 'Kapitel', 'Hinunter', 'in', 'Kaninchenbau', '.', 'Alice', 'fing'] -> Target (Y): den
Context (X): ['Kapitel', 'Hinunter', 'in', 'den', '.', 'Alice', 'fing', 'an'] -> Target (Y): Ka

### 1.4 Definition des Models
Als nächstes definieren wir unser Model. Dazu verwenden wir die Sequential API von Keras. Dieser [Blogpost](https://lilianweng.github.io/lil-log/2017/10/15/learning-word-embedding.html) bietet nochmal eine anschauliche Erklärung, wie word2vec CBOW funktioniert.
Das folgende Bild ist daraus und stellt die Architektur des Neuronalen Netzes dar:
<img src="https://lilianweng.github.io/lil-log/assets/images/word2vec-cbow.png" width=500 />

Weitere Informationen über den Embedding Layer finden sich [hier](https://keras.io/layers/embeddings/).

> **Tipp:** Nehmt euch wirklich Zeit Embeddings zu verstehen.

In [18]:
import tensorflow as tf
tf.keras.losses.CategoricalCrossentropy()

In [22]:
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda

#Modelldefinition
cbow = Sequential()
cbow.add(Embedding(input_dim=vocabulary_size, output_dim=100, input_length=window_size*2))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(100,)))
cbow.add(Dense(4400, activation='softmax'))
cbow.compile(loss='categorical_crossentropy', optimizer='rmsprop')


# Zusammenfassung
print(cbow.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 4, 100)            440000    
                                                                 
 lambda_1 (Lambda)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 4400)              444400    
                                                                 
Total params: 884,400
Trainable params: 884,400
Non-trainable params: 0
_________________________________________________________________
None


### 1.5 Training
Jetzt wird es ernst: Wir trainieren unser Modell.
Da wir eine eigene Generatorfunktion verwenden, müssen wir `steps_per_epoch` angeben. Überlegt euch was damit genau gemeint ist, wofür uns das nützt und was wir bei der Berechnung beachten müssen. Tipp: das Modell sollte pro Epoche alle Trainingsdaten sehen. Überlegt auch ob die Cbow-Fenstergröße einen einfluss auf diese Anzahl hat.

Weil wir unser Modell gerne abspeichern möchten, zum Beispiel, um es später weiter zu trainieren, definieren wir eine Callback-Funktion, die das für uns übernimmt.

In [23]:
from keras.callbacks import ModelCheckpoint

model_checkpoint = ModelCheckpoint('embeddings.hd5', monitor='loss', verbose=1, save_best_only=True, save_weights_only=False)

In [24]:
epochs = 5 #(kann gerne erhöht werden)
batch_size = 100
# das Modell sollte pro Epoche alle Trainingsdaten sehen. Überlegt auch ob die Cbow-Fenstergröße einen einfluss auf diese Anzahl hat.
total_len = len([item for sublist in numeric_docs for item in sublist])
steps_per_epoch = np.ceil(total_len / batch_size)

cbow.fit(generate_context_word_batches(numeric_docs, 4, vocabulary_size, batch_size), callbacks=[model_checkpoint], epochs=epochs, steps_per_epoch=steps_per_epoch)    

vocab size: 4400
length of corpus after flatten: 31916
Epoch 1/5
315/320 [============================>.] - ETA: 0s - loss: 7.3386
Epoch 1: loss improved from inf to 7.32760, saving model to embeddings.hd5
INFO:tensorflow:Assets written to: embeddings.hd5\assets
320/320 [==============================] - 4s 9ms/step - loss: 7.3276
Epoch 2/5
312/320 [============================>.] - ETA: 0s - loss: 6.3816
Epoch 2: loss improved from 7.32760 to 6.38284, saving model to embeddings.hd5
INFO:tensorflow:Assets written to: embeddings.hd5\assets
320/320 [==============================] - 3s 9ms/step - loss: 6.3828
Epoch 3/5
320/320 [==============================] - ETA: 0s - loss: 6.2903
Epoch 3: loss improved from 6.38284 to 6.29030, saving model to embeddings.hd5
INFO:tensorflow:Assets written to: embeddings.hd5\assets
320/320 [==============================] - 3s 8ms/step - loss: 6.2903
Epoch 4/5
319/320 [============================>.] - ETA: 0s - loss: 6.2222
Epoch 4: loss improved from

### 1.6 Test
Nachdem wir unser Modell nur sehr kurz und nur auf wenigen Daten trainiert haben, ist davon auszugehen, dass die Ergebnisse nicht optimal sind. Einen kurzen Blick wollen wir dennoch riskieren.

Dazu extrahieren wir zunächst die Gewichte aus dem Embedding-Layer und schauen sie uns auszugsweise an.

In [32]:
import pandas as pd
from keras.models import load_model

cbow = load_model('.\embeddings.hd5')
embedding_weights = cbow.layers[0].get_weights()[0]

embeddings = pd.DataFrame(embedding_weights, index=list(id2word.values()))
print(embeddings.head())

Da durch scharfes Hinsehen nicht unmittelbar zu erkennen ist, wie gut unsere Embeddings schon sind, machen wir stichprobenartige Tests. Dazu wählen wir einige Wörter und berechnen für deren Embeddings die Ähnlichkeit mit allen anderen Embedding-Vektoren in unserer Gewichtsmatrix. Anschließend lassen wir uns die fünf ähnlichsten Wörter ausgeben.
Überlegt euch welches Distantzmaß für den Vergleich von Vektoren genutzt werden kann. Tipp: Die Cosinusähnlichkeit könnte damit was zu tun haben. 

In [46]:
#Tipp: from sklearn.metrics.pairwise import ?
from numpy.linalg import norm

sample_terms = ['Alice', 'Hut', 'Kaninchen','Kaninchenbau']
sample_embeddings = embeddings.loc[sample_terms]


A = embeddings.to_numpy()

# Berechne die paarweisen Distanzen zwischen Beispielwörtern und Gesamtvokabular
distance_matrix = [ (np.dot(A, embeddings.loc[term].to_numpy())/(norm(A, axis=1)*norm(embeddings.loc[term].to_numpy()))) for term in sample_terms]
sample_embeddings.head()

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
Alice,-0.020176,0.068234,-0.022981,-0.113925,0.000010,-0.060666,0.033836,0.073994,-0.082541,0.167067,...,0.039249,0.087949,0.014285,0.143929,-0.028709,0.021554,0.042838,0.100621,-0.031212,-0.117024
Hut,0.005871,-0.014835,-0.005536,0.019242,-0.005796,0.010178,-0.003519,0.063346,-0.022088,0.009534,...,0.040193,-0.031693,-0.056136,-0.025825,-0.042177,0.048348,0.048588,0.000376,-0.054195,-0.034273
Kaninchen,-0.016645,-0.130349,-0.090484,-0.077052,-0.095148,-0.097698,0.061839,0.025079,-0.147395,0.059099,...,0.075458,0.120343,-0.129861,0.117600,-0.085283,0.018889,0.030357,0.084628,-0.108566,-0.089102
Kaninchenbau,-0.070573,-0.015516,-0.030562,0.004629,-0.036621,-0.075569,0.053498,0.035982,-0.024312,0.084775,...,0.051467,0.088120,0.006619,0.086973,0.022594,0.080801,0.064742,0.055466,-0.054508,-0.045317


In [47]:
# Zeige die top fünf ähnlichsten Wörter zu unseren Beispielwörtern 
similar_words = {sample_term: [id2word[idx] for idx in distance_matrix[index].argsort()[1:6]] 
                   for index, sample_term in enumerate(sample_terms)}

similar_words

{'Alice': ['Geschöpfen', 'persönliche', 'unwissendes', 'eos', 'Stunde'],
 'Hut': ['unwissendes', 'Bekanntschaft', 'Fläschchens', 'Stunde', 'dessen'],
 'Kaninchen': ['euch?«', 'eos', 'liebevolle', 'unwissendes', 'Der'],
 'Kaninchenbau': ['euch?«', 'unwissendes', 'Stunde', 'spürte', 'all']}

Seid ihr zufrieden mit den ähnlichen Wörtern? Woran kann es liegen, dass die Wörter nicht immer unbedingt Sinn ergeben?

## Aufgabe 2: Gensim als Wrapper für Word2Vec-Modelle
Embedding-Layer begegnen einem in der Praxis in der Tat häufig. In der Regel aber nicht als Bestandteile von reinen Word Embedding-Trainingsmodellen, sondern als erster Layer für Modelle mit anderen Aufgaben. Die Embeddings werden dann entweder mit vorberechneten Werten initalisiert und/oder werden im Training des Modells für die Downstream-Aufgabe (Textklassifikation, Übersetzung, ...) mittrainiert.

Eine komfortable Möglichkeit, eigene Word2Vec-Modelle zu trainieren, bietet [Gensim](https://radimrehurek.com/gensim/models/word2vec.html), eine Bibliothek, die für diese Modelle auch Wrapper bereitstellt, um komfortabler an die Embeddings zu kommen und mit diesen zu arbeiten.

Wir wollen uns im Folgenden einen kleinen Ausschnitt der Möglichkeiten, die Gensim bietet, anschauen.

### 2.0 Vorbereitung
Wir werden mit vortrainierten Google-News-Embeddings arbeiten, die ihr mit dem Code in den nächsten beiden Zellen herunterladen könnt. Falls das nicht funktionieren sollte, findet ihr die vortrainierten Embeddings [hier](https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit?usp=sharing). 

### 2.1 Word2Vec-Model laden
Die Vektoren haben eine Länge vin 300.
Zieht euch die Embeddings über den oben angegebenen Link und verwendet gensim, um sie anschließend zu laden.

**Hinweis**: Es kann einen Moment dauern, bis das Dictionary, das von Wort auf Embedding abbildet, erzeugt ist. Im Zweifel ein ```limit``` angeben und nur die ersten 1,5 Mio. Embeddings laden.

In [1]:
import gensim

embeddings = gensim.models.KeyedVectors.load_word2vec_format("./download/GoogleNews-vectors-negative300.bin", binary=True)

Aus Spaß an der Freude können wir nun schauen, wie gut unser trainiertes Modell ist bzw. ob es bestimmte von Menschen wahrgenommene Analogien bestätigt. 

Die Analogien sind [hier](https://github.com/nicholas-leonard/word2vec/blob/master/questions-words.txt) zu finden.

In [ ]:
from gensim.test.utils import datapath

embeddings.evaluate_word_analogies(datapath("questions-words.txt"), restrict_vocab=30000)

### 2.2 Spaß mit Semantik
Im Folgenden wollen wir uns mit dem Mehrwert beschäftigen, den semantische Embeddings bieten. Für weitere Inspiration siehe zum Beispiel [hier](https://www.machinelearningplus.com/nlp/gensim-tutorial/) und die [Doku](https://radimrehurek.com/gensim/models/keyedvectors.html).

Mit semantischen Vektoren lassen sich zum Beispiel folgende Fragen beantworten:


In [13]:
# Welche Stadt ist das New York Deutschlands? (Hinweis: 'New_York' ist als Token in den Embeddings enthalten)
print(f"Das deutsche New York ist: {embeddings.most_similar(positive=['New_York', 'German'])}\n")

# Was ist Emacs besonders ähnlich?
print(f"Ähnlichste Begriffe zu 'Emacs': {embeddings.most_similar(positive=['Emacs'])}\n")

# Und wie sieht es mit Vim aus?
print(f"Ähnlichste Begriffe zu 'Vim': {embeddings.most_similar(positive=['Vim'])}\n")

# Wer ist eigentlich der Mozart der Naturwissenschaft?
print(f"Der Mozart der Naturwissenschaft ist: {embeddings.most_similar(positive=['Science', 'Mozart'], negative=['Music'])}\n")

# Welches Wort verhält sich zu 'singing' wie 'burnt' zu 'burning'?
print(f"burning:burnt wie singing:{embeddings.similar_by_word('singing')}\n")

# Sind sich Deutschland und Frankreich ähnlicher oder Deutschland und Kanada?
print(f"Ähnlichkeit DE, FR: {embeddings.similarity('Germany', 'France')}")
print(f"Ähnlichkeit DE, CAN: {embeddings.similarity('Germany', 'Canada')}")

Das deutsche New York ist: [('Berlin', 0.6261184215545654), ('Austrian', 0.6248041391372681), ('Germany', 0.5939319133758545), ('Hamburg', 0.5871230363845825), ('Cologne', 0.580512285232544), ('Munich', 0.5640348196029663), ('Frankfurt', 0.5576122403144836), ('Hungarian', 0.5553871989250183), ('Manhattan', 0.5439103245735168), ('Budapest', 0.5419133901596069)]

Ähnlichste Begriffe zu 'Emacs': [('emacs', 0.7516424059867859), ('GNU_Emacs', 0.6907742023468018), ('wget', 0.6600909233093262), ('TextMate', 0.6504663825035095), ('Gnumeric', 0.6487864851951599), ('Notepad_+', 0.646020770072937), ('Win##', 0.6429656744003296), ('debian', 0.6395151615142822), ('libc', 0.6370412707328796), ('osx', 0.6365400552749634)]

Ähnlichste Begriffe zu 'Vim': [('Emacs', 0.4973173439502716), ('Menthe', 0.47791504859924316), ('deo', 0.47666671872138977), ('emacs', 0.46879321336746216), ('TextWrangler', 0.46436330676078796), ('Fantastik', 0.4486047923564911), ('TextMate', 0.44810065627098083), ('Chill_Pill', 0

Bei der Interpretation der Ergebnisse ist jedoch Vorsicht geboten: Es werden zwar semantische Beziehungen abgebildet, aber die entsprechen möglicherweise nicht immer den Erwartungen.

Wie ähnlich sind sich zum Beispiel "Leben" und "Tod", "kalt" und "warm", "Norden" und "Süden"?

Sind die Ergebnisse wie erwartet? Warum (nicht)?

In [16]:
print(f"Ähnlichkeit Leben, Tod: {embeddings.similarity('Life', 'Death')}")
print(f"Ähnlichkeit kalt, warm: {embeddings.similarity('Cold', 'Warm')}")
print(f"Ähnlichkeit Norden, Süden: {embeddings.similarity('north', 'south')}")


Ähnlichkeit Leben, Tod: 0.3389456570148468
Ähnlichkeit kalt, warm: 0.49793362617492676
Ähnlichkeit Norden, Süden: 0.9674535989761353


Die Embeddings sind nicht neutral, sondern spiegeln die Beziehungen wieder, die sich in den Trainingsdaten finden lassen.

In [26]:
# Wird Wissenschaft von Frauen oder Männern gemacht?
print(f"Wissenschaft wird gemacht von Männern: {embeddings.similarity('Science', 'Man')}")
print(f"Wissenschaft wird gemacht von Frauen: {embeddings.similarity('Science', 'Woman')}")
# Sind Mörder eher Schwarze, Weiße oder Asiaten?
print(f"Mörder sind Black: {embeddings.similarity('murderer', 'Black')}")
print(f"Mörder sind White: {embeddings.similarity('murderer', 'White')}")
print(f"Mörder sind Asian: {embeddings.similarity('murderer', 'Asian')}")
# Was bleibt vom Mann, wenn die Intelligenz abgezogen wird?
print(f"Mann ohne Intelligenz: {embeddings.most_similar(positive=['men'], negative=['intelligence'])[0]}")

Wissenschaft wird gemacht von Männern: 0.07765393704175949
Wissenschaft wird gemacht von Frauen: 0.15121036767959595
Mörder sind Black: 0.10026663541793823
Mörder sind White: 0.044914063066244125
Mörder sind Asian: -0.026033801957964897
Mann ohne Intelligenz: ('women', 0.5342543125152588)
